<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/03_advanced/graph_convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly-2.0-preview

    100% |████████████████████████████████| 75.4MB 478kB/s 
    100% |████████████████████████████████| 3.2MB 10.7MB/s 
    100% |████████████████████████████████| 61kB 26.8MB/s 
    100% |████████████████████████████████| 307kB 15.0MB/s 


In [0]:
import math
import tensorflow as tf

In [0]:
class GraphConvolution(tf.keras.Model):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        stdv = 1. / math.sqrt(out_features)
        self.weight = tf.Variable(
            tf.random.uniform(shape=[in_features, out_features],
                              minval=-stdv,
                              maxval=stdv)
        )
        
        if bias:
            self.bias = tf.Variable(
                tf.random.uniform(shape=[out_features],
                                  minval=-stdv,
                                  maxval=stdv)
            )
        else:
            self.bias = None

    def forward(self, input, adj):
        support = tf.matmul(input, self.weight)
        output = tf.sparse.sparse_dense_matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [0]:
layer = GraphConvolution(10, 5)

In [0]:
class GCN(tf.keras.Model):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = tf.keras.layers.Dropout(dropout)

    def forward(self, x, adj, training=False):
        x = tf.relu(self.gc1(x, adj))
        x = self.dropout(x, self.dropout, training=training)
        x = self.gc2(x, adj)
        return tf.nn.log_softmax(x, axis=1)

In [0]:
model = GCN(10, 50, 2, 0.3)